# PSU efficiency analysis

Perform all the PSU analysis presented in the IMC'25 paper (Section 9)

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

import plotly.graph_objects as go

## Initialization

In [ ]:

# Select the different output format settings

PaperPlot = True
# PaperPlot = False
if PaperPlot:
    output_format = 'IMC'
else:
    output_format = 'online'

if output_format == 'online':
    font_size_px = 14
    linewidth_px = 512
    landscapewidth_px = 654
    plot_path = None
    
    plot_path = Path('plots')

if output_format == 'IMC':
    font_size_pt = 7
    offset = 5 # to compensate for the rounding of unit conversions
    linewidth_pt = 241 - offset  
    landscapewidth_pt = 506 - offset
    
    # 1pt = 1.333px
    font_size_px = int(font_size_pt*1.333)+1
    linewidth_px = int(linewidth_pt*1.333)+1
    landscapewidth_px = int(landscapewidth_pt*1.333)+1

    out_path = Path('output/2025_IMC/figures')

# Create the output directory if don't exist
Path(out_path).mkdir(parents=True, exist_ok=True)

# Input data
input_path = Path('input')

# Default plot layout
default_layout = {
    "title":None,
    "width":linewidth_px,
    "height":200,
    "font":{"size":font_size_px},
    "yaxis":{'title':{'font':{'size':font_size_px}}},
    "xaxis":{'title':{'font':{'size':font_size_px},
                      'text':'Time [ s ]'}}
}


## Efficient curve and 80 Plus standards

In [ ]:
# ===============
# Import PFE data and derive the numerical models 
# for the different standard by adding a constant 
# to the PFE efficiency curve
# ===============

pfe_file = Path(input_path,'pfe-efficiency-curve.csv')
df_pfe = pd.read_csv(pfe_file)
df_pfe['load_%'] = (df_pfe['load_W'] / 600)*100

# .. Model is linear interpolation between the colected points
def eff_interp(x):
    return np.interp(x,df_pfe['load_%']/100,df_pfe['efficiency_%']/100)

marker_opacity = 1
marker_size = 9
marker_border_size = 1
marker_line = dict(
    width=marker_border_size,
    color='black'
)

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = [10,20,50,100],
        y = [90,94,96,91],
        mode='markers',
        name='Titanium',
        marker=dict(
            symbol=19, 
            line=marker_line,
            size=marker_size, 
            color='rgb(121, 121, 130)', 
            ),
    )
)
fig.add_trace(
    go.Scatter(
        x = [20,50,100],
        y = [90,94,91],
        mode='markers',
        name='Platinum',
        marker=dict(symbol=0, 
            line=marker_line,size=marker_size, color='rgb(209, 208, 206)', ),
    )
)
fig.add_trace(
    go.Scatter(
        x = [20,50,100],
        y = [88,92,88],
        mode='markers',
        name='Gold',
        marker=dict(symbol=2,
            line=marker_line, size=marker_size, color='rgb(255, 215, 0)', opacity=marker_opacity),
    )
)
fig.add_trace(
    go.Scatter(
        x = [20,50,100],
        y = [85,89,85],
        mode='markers',
        name='Silver',
        marker=dict(symbol=4,
            line=marker_line, size=marker_size, color='rgb(192, 192, 192)', opacity=marker_opacity),
    )
)
fig.add_trace(
    go.Scatter(
        x = [20,50,100],
        y = [81,85,81],
        mode='markers',
        name='Bronze',
        marker=dict(symbol=3,
            line=marker_line, size=marker_size, color='rgb(205, 127, 50)', opacity=marker_opacity),
    )
)

fig.add_trace(
    go.Scatter(
        x = df_pfe['load_%'],
        y = df_pfe['efficiency_%'],
        mode='lines',
        name='PFE600',
        marker=dict(
            color='black', 
        )
    )
)

 # y axis title
ytitle = go.layout.Annotation(
        x=-0.01,
        y=1.15,
        xref="paper",
        yref="paper",
        text="Efficiency (%)",
        showarrow=False,
        xanchor='left'
    )

# Define the custom layout options
custom_layout = dict(
    xaxis=dict(
        title=dict(
            text='Power load (%)',
            font={'size':font_size_px}
        ),
        range=[-5,105],
    ),
    yaxis=dict(
        title=None,
        range=[80,100],
    ),
    legend=dict(
        orientation='v'
    ),
    annotations=[ytitle],
    margin=dict(l=0, r=0, t=25, b=0),
)
# Combine with the defaults and apply
layout = default_layout.copy()
layout.update(custom_layout)
fig.update_layout(layout)

fig.show()
fig.write_image(out_path/'80Plus-curves.pdf')

In [ ]:
# ===============
# Interpolation functions for the different 80Plus standards
# ===============

# Function f(x) for 80PLUS Titanium standard
def t(x):
    const = 0.94 - eff_interp(0.20) + 0.02
    return eff_interp(x) + const
    # return -0.2083 * x**2 + 0.2125 * x + 0.90583

# Function f(x) for 80PLUS Platinum standard
def p(x):
    const = 0
    return eff_interp(x) + const
    # return -0.2417 * (x)**2 + 0.3025 * (x) + 0.84917

# Function f(x) for 80PLUS Gold standard
def g(x):
    const = 0.92 - eff_interp(0.50)
    return eff_interp(x) + const
    # return -0.2667 * (x)**2 + 0.32 * (x) + 0.8267

# Function f(x) for 80PLUS Silver standard
def s(x):
    const = 0.89 - eff_interp(0.50)
    return eff_interp(x) + const
    # return -0.2667 * (x)**2 + 0.32 * (x) + 0.7967

# Function f(x) for 80PLUS Bronze standard
def b(x):
    const = 0.85 - eff_interp(0.50)
    return eff_interp(x) + const
    # return -0.2667 * (x)**2 + 0.32 * (x) + 0.7567

# Function f(x) for an 80PLUS-like standard
# with custom constent term
def custom_eff(x,const):
    return eff_interp(x) + const

In [ ]:
# ===============
# Load the PSU efficiency data
# ===============

def load_src_data():
        
    src_file = Path(input_path,'psu-efficiency.csv')
    df = pd.read_csv(src_file)

    df['load_PSU1'] = df['median_power_PSU1'].div(df['PSU_capacity'])
    df['load_PSU2'] = df['median_power_PSU2'].div(df['PSU_capacity'])

    return df

df = load_src_data()

df


## Plot eff=f(load)

In [ ]:
# ===============
# Plot the PSU conversion efficiency data
# ===============

df = load_src_data()

fig = go.Figure()

marker_opacity = 0.5
marker_size = 9
marker_border_size = 1
marker_line = dict(
    width=marker_border_size,
    color='black'
)

# Combine the data from both PSU
loads = df['load_PSU1'].tolist()
loads.append(df['load_PSU2'].tolist())
efficiencies = df['efficiency_PSU1'].tolist()
efficiencies.append(df['efficiency_PSU2'].tolist())


fig.add_trace(
    go.Scatter(
        x = df_pfe['load_%']/100,
        y = df_pfe['efficiency_%']/100 + 0.94 - eff_interp(0.20) + 0.02,
        mode='lines',
        name='Titanium',
        marker=dict(
            color='rgb(121, 121, 130)', 
        ),
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x = [0.2],
        y = [0.96],
        mode='markers',
        name='Titanium',
        marker=dict(
            symbol=19, 
            line=marker_line,
            size=marker_size, 
            color='rgb(121, 121, 130)', 
            ),
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x = df_pfe['load_%']/100,
        y = df_pfe['efficiency_%']/100,
        mode='lines',
        name='Platinum',
        marker=dict(
            color='rgb(209, 208, 206)', 
        ),
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x = [0.2],
        y = [0.922],
        mode='markers',
        name='Platinum',
        marker=dict(symbol=0, 
            line=marker_line,size=marker_size, color='rgb(209, 208, 206)', ),
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x = df_pfe['load_%']/100,
        y = df_pfe['efficiency_%']/100 + 0.92 - eff_interp(0.50),
        mode='lines',
        name='Gold',
        marker=dict(
            color='rgb(255, 215, 0)', 
        ),
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x = [0.20],
        y = [0.899],
        mode='markers',
        name='Gold',
        marker=dict(symbol=2,
            line=marker_line, size=marker_size, color='rgb(255, 215, 0)', 
            ),
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x = df_pfe['load_%']/100,
        y = df_pfe['efficiency_%']/100 + 0.89 - eff_interp(0.50),
        mode='lines',
        name='Silver',
        marker=dict(
            color='rgb(192, 192, 192)', 
        ),
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x = [0.20],
        y = [0.87],
        mode='markers',
        name='Silver',
        marker=dict(symbol=4,
            line=marker_line, size=marker_size, color='rgb(192, 192, 192)', 
            ),
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x = df_pfe['load_%']/100,
        y = df_pfe['efficiency_%']/100 + 0.85 - eff_interp(0.50),
        mode='lines',
        name='Bronze',
        marker=dict(
            color='rgb(205, 127, 50)', 
        ),
        showlegend=False
    )
)
fig.add_trace(
    go.Scatter(
        x = [0.20],
        y = [0.83],
        mode='markers',
        name='Bronze',
        marker=dict(symbol=3,
            line=marker_line, size=marker_size, color='rgb(205, 127, 50)', 
            ),
        showlegend=False
    )
)


fig.add_trace(
    go.Scatter(
        x = loads,
        y = efficiencies,
        mode='markers',
        marker=dict(
            opacity=marker_opacity,
        ),
        showlegend=False
    )
)


 # y axis title
ytitle = go.layout.Annotation(
        x=-0.01,
        y=1.15,
        xref="paper",
        yref="paper",
        text="Efficiency (%)",
        showarrow=False,
        xanchor='left'
    )

# Define the custom layout options
custom_layout = dict(
    xaxis=dict(
        title=dict(
            text='Power load (%)',
            font={'size':font_size_px}
        ),
        range=[0.03,0.23],
        tickmode = 'array',
        tickvals = [0.05, 0.1, 0.15, 0.2],
        ticktext = [5, 10, 15, 20]
    ),
    yaxis=dict(
        title=None,
        range=[0.50,1.05],
    ),
    legend=dict(
        orientation='v'
    ),
    width=0.3*landscapewidth_px,
    annotations=[ytitle],
    margin=dict(l=0, r=0, t=25, b=0),
)
# Combine with the defaults and apply
layout = default_layout.copy()
layout.update(custom_layout)
fig.update_layout(layout)


fig.show()
fig.write_image(out_path/'efficiency_all.pdf')

In [ ]:
# ===============
# Highlight the data from a given router model
# ===============


# Filter for the router model of interest

# model_id = '8201-32FH'
# model_id = 'NCS-55A1-24H'
model_id = 'ASR-920-24SZ-M'
# model_id = '8201-24H8FH'
tmp = df.loc[df['router_model']== model_id]

# Combine the data from both PSU
loads = tmp['load_PSU1'].tolist()
loads.append(tmp['load_PSU2'].tolist())
efficiencies = tmp['efficiency_PSU1'].tolist()
efficiencies.append(tmp['efficiency_PSU2'].tolist())

fig.add_trace(
    go.Scatter(
        x = loads,
        y = efficiencies,
        mode='markers',
        marker=dict(
            color='red', 
            opacity=marker_opacity,
        ),
        showlegend=False
    )
)

# Define the custom layout options
custom_layout = dict(
    width=0.2*landscapewidth_px,
)
# Combine with the defaults and apply
# layout = default_layout.copy() <- Don't copy! We want to update from the previous plot
layout.update(custom_layout)
fig.update_layout(layout)

fig.write_image(out_path/str('efficiency_'+model_id+'.pdf'))

fig.show()

## How much would we save with better PSUs?

In [ ]:
df = load_src_data()
tex_output = ''

# Compute the theoretic efficiency for different standards
for standard in [b,s,g,p,t]:
    for PSU in ['1','2']:
        # .. compute the efficiency
        label_eff = 'efficiency_PSU' + PSU + '_' + standard.__name__
        df[label_eff] = df['load_PSU' + PSU].apply(standard)
        df[label_eff] = df[[label_eff,'efficiency_PSU' + PSU]].max(axis=1)
        
        # .. derive the resulting power draw
        label_power = 'median_power_PSU' + PSU + '_' + standard.__name__
        df[label_power] = df['median_power_PSU' + PSU] * df['efficiency_PSU' + PSU] / df[label_eff]

        # .. compute the correspinding savings
        label_savings_abs = 'savings_abs_PSU' + PSU + '_' + standard.__name__
        label_savings_rel = 'savings_rel_PSU' + PSU + '_' + standard.__name__
        df[label_savings_abs] = df['median_power_PSU' + PSU] - df[label_power] 
        df[label_savings_rel] = df[label_savings_abs] / df['median_power_PSU' + PSU] 

    # .. Compute the gains
    savings_total_abs = df['savings_abs_PSU1_'+standard.__name__].sum(axis=0) +  df['savings_abs_PSU2_'+standard.__name__].sum(axis=0) 
    savings_total_rel = savings_total_abs / (df['median_power_PSU1'].sum(axis=0) + df['median_power_PSU2'].sum(axis=0))

    print('For',standard.__name__)
    print("{}\\% ({} W)\n".format(
        int(100*savings_total_rel),
        int(savings_total_abs)
        ))
    
    tex_output += '& {}\\% ({} W)'.format(
        int(100*savings_total_rel),
        int(savings_total_abs)
        )
tex_output += '\\\\'
df

print(tex_output)

## How much would we save with better-sized PSUs?

In [ ]:
df = load_src_data()
psu_capacities = sorted(df['PSU_capacity'].unique())

pd.set_option('display.max_columns', None)

overprovision_factor = [1,2,3]

def set_min_capacity(row, capacity_options, overprovision_factor):
    for c in sorted(capacity_options):
        if ((row['median_power_PSU1'] > c/overprovision_factor) or
            (row['median_power_PSU2'] > c/overprovision_factor)):
            continue
        else:
            return c
        
def set_new_capacity(row, capacity):
    if (row['min_capacity'] > capacity):
        return row['min_capacity']
    else:
        return capacity
    
def get_new_efficiency(row, PSU, label_load):
    offset = row['efficiency_PSU' + PSU] - eff_interp(row['load_PSU' + PSU])
    return custom_eff(row[label_load],offset)
    
for k in overprovision_factor:

    tex_output = ''
    print('k = {}'.format(k))
    
    # .. define the minimal capacity required per router
    df['min_capacity'] = df.apply(set_min_capacity, args=(psu_capacities, k), axis=1)

    # .. set the different cases of smallest considered capacities
    for capacity in psu_capacities:

        # .. compute the min capacity per router per case
        label_capacity = 'capacity_'+str(capacity)+'+'
        df[label_capacity] = df.apply(set_new_capacity, args=(capacity, ), axis=1)
        
        for PSU in ['1','2']:
            # .. compute the corresponding load
            label_load = 'load_PSU' + PSU + '_' + label_capacity
            df[label_load] = df['median_power_PSU' + PSU] / df[label_capacity]
            # .. compute the corresponding efficiency
            label_eff = 'efficiency_PSU' + PSU + '_' + label_capacity
            df[label_eff] = df.apply(get_new_efficiency, args=(PSU,label_load,), axis=1)

            # .. derive the resulting power draw
            label_power = 'median_power_PSU' + PSU + '_' + label_capacity
            df[label_power] = df['median_power_PSU' + PSU] * df['efficiency_PSU' + PSU] / df[label_eff]

            # .. compute the correspinding savings
            label_savings_abs = 'savings_abs_PSU' + PSU + '_' + label_capacity
            label_savings_rel = 'savings_rel_PSU' + PSU + '_' + label_capacity
            df[label_savings_abs] = df['median_power_PSU' + PSU] - df[label_power] 
            df[label_savings_rel] = df[label_savings_abs] / df['median_power_PSU' + PSU] 



        # .. Compute the total gains
        savings_total_abs = df['savings_abs_PSU1_' + label_capacity].sum(axis=0) +  df['savings_abs_PSU2_' + label_capacity].sum(axis=0) 
        savings_total_rel = savings_total_abs / (df['median_power_PSU1'].sum(axis=0) + df['median_power_PSU2'].sum(axis=0))

        print('For $',label_capacity,'$')
        print("{}\\% ({} W)\n".format(
            int(savings_total_rel*100),
            int(savings_total_abs)
            ))
        

        tex_output += '& {}\\% ({} W)'.format(
                int(100*savings_total_rel),
                int(savings_total_abs)
                )

    tex_output += '\\\\'
    print(tex_output)

    display(df)

## How much would we save by loading only one PSU instead of two?

In [ ]:
# Load the data
df = load_src_data()
tex_output = ''

# .. compute the total load
df['total_power_out'] = df['median_power_PSU1']*df['efficiency_PSU1'] + df['median_power_PSU2']*df['efficiency_PSU2']
df['total_load'] = df['total_power_out'].div(df['PSU_capacity'])

def get_new_efficiency(row, PSU, label_load):
    offset = row['efficiency_PSU' + PSU] - eff_interp(row['load_PSU' + PSU])
    return custom_eff(row[label_load],offset)

# .. compute the efficiency for that load if running on only one of the PSUs
for PSU in ['1','2']:
    label_eff = 'efficiency_PSU' + PSU + '_total_load' 
    df[label_eff] = df.apply(get_new_efficiency, args=(PSU,'total_load',), axis=1)
# .. take the max
df['max_efficiency'] = df[['efficiency_PSU1_total_load','efficiency_PSU2_total_load']].max(axis=1)
# .. get the resulting total power in
df['total_power_in'] = df['total_power_out'] / df['max_efficiency']

# .. compute the savings per router
df['savings_abs'] = (df['median_power_PSU1']+ df['median_power_PSU2']) - df['total_power_in']
df['savings_rel'] = df['savings_abs'] / (df['median_power_PSU1']+ df['median_power_PSU2'])

# .. compute the global savings
savings_total_abs = df['savings_abs'].sum(axis=0)
savings_total_rel = savings_total_abs / (df['median_power_PSU1'].sum(axis=0) + df['median_power_PSU2'].sum(axis=0))


print('For using only one PSU')
print("{}\\% ({} W)\n".format(
    int(savings_total_rel*100),
    int(savings_total_abs)
    ))

tex_output += '& {}\\% ({} W)'.format(
        int(100*savings_total_rel),
        int(savings_total_abs)
        )

print(tex_output)

df

## How much would we save by loading only one PSU instead of two AND that one be of better quality?

In [ ]:
# [continuing from the previous df]
tex_output = ''

# Compute the theoretic efficiency for different standards
for standard in [b,s,g,p,t]:
    # .. compute the efficiency
    label_eff = 'efficiency_' + standard.__name__
    df[label_eff] = df['total_load'].apply(standard)
    df[label_eff] = df[[label_eff,'max_efficiency']].max(axis=1)

    # .. derive the resulting power draw
    label_power = 'median_power_' + standard.__name__
    df[label_power] = df['total_power_in'] * df['max_efficiency'] / df[label_eff]

    # .. compute the savings per router
    label_savings_abs = 'savings_abs_' + standard.__name__
    label_savings_rel = 'savings_rel_' + standard.__name__
    df[label_savings_abs] = df['total_power_in'] - df[label_power] 
    df[label_savings_rel] = df[label_savings_abs] / df['total_power_in'] 

    # .. compute the global savings
    print('For using only one PSU of at least standard ',standard.__name__, '\\\\')

    # .. 1. from the first step
    savings_total_abs = df['savings_abs'].sum(axis=0)
    savings_total_rel = savings_total_abs / (df['median_power_PSU1'].sum(axis=0) + df['median_power_PSU2'].sum(axis=0))
    print("using only one {}\\% ({} W)\\\\".format(
    int(100*savings_total_rel),
    int(savings_total_abs)
    ))
    
    # .. 2. from this step alone
    savings_total_abs = df['savings_abs_'+standard.__name__].sum(axis=0)
    savings_total_rel = savings_total_abs / (df['median_power_PSU1'].sum(axis=0) + df['median_power_PSU2'].sum(axis=0))
    print("using a better one {}\\% ({} W)\\\\".format(
    int(100*savings_total_rel),
    int(savings_total_abs)
    ))

    # .. 3. all together
    savings_total_abs = df['savings_abs_'+standard.__name__].sum(axis=0) + df['savings_abs'].sum(axis=0)
    savings_total_rel = savings_total_abs / (df['median_power_PSU1'].sum(axis=0) + df['median_power_PSU2'].sum(axis=0))
    print("total: {}\\% ({} W)\n".format(
    int(100*savings_total_rel),
    int(savings_total_abs)
    ))

    tex_output += '& {}\\% ({} W)'.format(
        int(100*savings_total_rel),
        int(savings_total_abs)
        )
    
tex_output += '\\\\'
print(tex_output)

df
